# Analysis of US Inflation: Trends in Food, Energy, and Housing Indices

**Importing Libraries & Datasets**

In [13]:
import pandas as pd

# Load Food at Home CPI
food_df = pd.read_csv(r"D:\Masters Study Abroad\BA 2nd Sem\IS6611 Applied Research in Business Analytics\IS6611 IT Artefact V3\Final Files\Final Datasets\US Inflation Raw\US_Food_Index.csv", skiprows=11)
# Load Energy CPI
energy_df = pd.read_csv(r"D:\Masters Study Abroad\BA 2nd Sem\IS6611 Applied Research in Business Analytics\IS6611 IT Artefact V3\Final Files\Final Datasets\US Inflation Raw\US_Energy_Index.csv", skiprows=11)
# Load Housing CPI
housing_df = pd.read_csv(r"D:\Masters Study Abroad\BA 2nd Sem\IS6611 Applied Research in Business Analytics\IS6611 IT Artefact V3\Final Files\Final Datasets\US Inflation Raw\US_Housing_Index.csv", skiprows=11)

**Data Preparation and Restructuring**

In [14]:
# List of month columns to keep
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Melt function to reshape each dataframe
food_long = pd.melt(housing_df[['Year'] + months], id_vars='Year', 
                    value_vars=months, var_name='Month', value_name='Food_Index')
energy_long = pd.melt(energy_df[['Year'] + months], id_vars='Year', 
                      value_vars=months, var_name='Month', value_name='Energy_Index')
housing_long = pd.melt(food_df[['Year'] + months], id_vars='Year', 
                       value_vars=months, var_name='Month', value_name='Housing_Index')

# Mapping for month name to number
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 
    'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
    'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Add a proper datetime column
for df, name in [(food_long, 'Food'), (energy_long, 'Energy'), (housing_long, 'Housing')]:
    df['Month_Num'] = df['Month'].map(month_map)
    df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month_Num'])

# Drop unnecessary columns
food_long = food_long[['Date', 'Food_Index']]
energy_long = energy_long[['Date', 'Energy_Index']]
housing_long = housing_long[['Date', 'Housing_Index']]

**Dataset Inspection**

In [15]:
food_long.head()

,Date,Food_Index
0,2015-01-01,232.167
1,2016-01-01,236.811
2,2017-01-01,244.133
3,2018-01-01,251.133
4,2019-01-01,258.480


In [16]:
energy_long.head()

,Date,Energy_Index
0,2015-01-01,190.941
1,2016-01-01,178.755
2,2017-01-01,198.579
3,2018-01-01,209.700
4,2019-01-01,198.735


In [17]:
housing_long.head()

,Date,Housing_Index
0,2015-01-01,242.358
1,2016-01-01,240.924
2,2017-01-01,236.372
3,2018-01-01,238.966
4,2019-01-01,240.630


**Dataset Consolidation**

In [18]:
# Merging all three datasets on the 'Date' column
US_inflation_df = food_long.merge(energy_long, on='Date').merge(housing_long, on='Date')

# Sort by date
US_inflation_df.sort_values(by='Date', inplace=True)

# Reset index
US_inflation_df.reset_index(drop=True, inplace=True)

# Preview the table
US_inflation_df.head()

,Date,Food_Index,Energy_Index,Housing_Index
0,2015-01-01,232.167,190.941,242.358
1,2015-02-01,232.601,195.436,242.026
2,2015-03-01,232.900,204.335,240.896
3,2015-04-01,233.139,203.430,240.587
4,2015-05-01,233.512,214.808,240.200


**Computing Percentage Growth in Inflation Indices**

In [19]:
# Get the base (first row) values
base_food = US_inflation_df['Food_Index'].iloc[0]
base_energy = US_inflation_df['Energy_Index'].iloc[0]
base_housing = US_inflation_df['Housing_Index'].iloc[0]

# Calculate percentage changes
US_inflation_df['Food_Growth_%'] = ((US_inflation_df['Food_Index'] - base_food) / base_food * 100).round(2)
US_inflation_df['Energy_Growth_%'] = ((US_inflation_df['Energy_Index'] - base_energy) / base_energy * 100).round(2)
US_inflation_df['Housing_Growth_%'] = ((US_inflation_df['Housing_Index'] - base_housing) / base_housing * 100).round(2)

# Display updated DataFrame
US_inflation_df.head()

,Date,Food_Index,Energy_Index,Housing_Index,Food_Growth_%,Energy_Growth_%,Housing_Growth_%
0,2015-01-01,232.167,190.941,242.358,0.00,0.00,0.00
1,2015-02-01,232.601,195.436,242.026,0.19,2.35,-0.14
2,2015-03-01,232.900,204.335,240.896,0.32,7.01,-0.60
3,2015-04-01,233.139,203.430,240.587,0.42,6.54,-0.73
4,2015-05-01,233.512,214.808,240.200,0.58,12.50,-0.89


**Final Dataset Export**

In [20]:
# Export to CSV
US_inflation_df.to_csv(r"D:\Masters Study Abroad\BA 2nd Sem\IS6611 Applied Research in Business Analytics\IS6611 IT Artefact V3\Final Files\Final Datasets\US Inflation Raw\US_inflation_index.csv", index=False)